### 1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и 
реализовать функцию, записывающую собранные вакансии в созданную БД


In [1]:
from pymongo import MongoClient
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests

In [2]:
import re
import pandas as pd
import transliterate
import json

In [3]:
client  = MongoClient('localhost',27017)
db = client['vakans']
vakans_hh = db.vakans.vakans_hh
vakans_sj = db.vakans.vakans_sj

In [4]:
def page_table(link):
    result = pd.DataFrame()

    global_name = 'https://hh.ru'
    main_link_last = f'{global_name}{link}'
    response_last = requests.get(main_link_last, headers = header).text
    html_last = bs(response_last,'lxml')

    names = html_last.findAll('div',{'class':'resume-search-item__name'})
    names_list = []
    for i in names:
        names_list.append(i.getText())

    salarys = html_last.findAll('div',{'class':'vacancy-serp-item__row vacancy-serp-item__row_header'})
    salarys_all = []
    for i in salarys:
        salarys_2 = i.find('div',{'class':'vacancy-serp-item__compensation'})
        salarys_all.append(salarys_2)
    for i in range(len(salarys_all)):
        if salarys_all[i]:
            salarys_all[i] = salarys_all[i].getText()
    salarys_1 = []
    salarys_2 = []
    val = []
    for i in range(len(salarys_all)):
        salarys_1.append(None)
        salarys_2.append(None)
        val.append(None)
        if salarys_all[i]:
            hm = re.findall(r'^от_*', salarys_all[i])
            if hm:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                hm_num = re.sub(r'от ', '', hm_num)
                hm_val = re.search(r'[\D]+', hm_num)
                hm_num = re.sub(r'[\D]+', '', hm_num)
                salarys_1[i] = hm_num
                val[i] = hm_val[0]
            hm2 = re.findall(r'^до_*', salarys_all[i])
            if hm2:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                hm_num = re.sub(r'до ', '', hm_num)
                hm_val = re.search(r'[\D]+', hm_num)
                hm_num = re.sub(r'[\D]+', '', hm_num)
                salarys_2[i] = hm_num
                val[i] = hm_val[0]
            hm3 = re.search(r'-', salarys_all[i])
            if hm3:
                hm_num = re.split(r'-', salarys_all[i])
                hm_num1 = re.sub(r'\xa0', '', hm_num[0])
                hm_num2 = re.sub(r'\xa0', '', hm_num[1])
                hm_val = re.search(r'[\D]+', hm_num2)
                hm_num2 = re.sub(r'[\D]+', '', hm_num2)
                salarys_1[i] = hm_num1
                salarys_2[i] = hm_num2
                val[i] = hm_val[0]

    links_page = []
    for i in salarys:
        links_page1 = i.find('a',{'class':'bloko-link HH-LinkModifier'})
        links_page2 = links_page1['href']
        links_page.append(links_page2)
    
    result['names'] = names_list
    result['salarys_1'] = salarys_1
    result['salarys_2'] = salarys_2
    result['val'] = val
    result['links'] = links_page
    result['from'] = global_name
    
    return result

In [5]:
def page_table_two(link):
    result = pd.DataFrame()

    global_name = 'https://superjob.ru'
    main_link_last = f'{global_name}{link}'
    response_last = requests.get(main_link_last, headers = header).text
    html_last = bs(response_last,'lxml')
    
    names = html_last.findAll('div',{'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    names_list = []
    for i in names:
        names_list.append(i.getText())

    salarys_all = html_last.findAll('span',{'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
    for i in range(len(salarys_all)):
        salarys_all[i] = salarys_all[i].getText()   
    
    salarys_1 = []
    salarys_2 = []
    val = []
    for i in range(len(salarys_all)):
        hm = re.findall(r'^от_*', salarys_all[i])
        hm2 = re.findall(r'^до_*', salarys_all[i])
        hm3 = re.search(r'—', salarys_all[i])
        hm4 = re.search(r'По договорённости', salarys_all[i])
        if hm:
            hm_num = re.sub(r'\xa0', '', salarys_all[i])
            hm_num = re.sub(r'от', '', hm_num)
            hm_val = re.search(r'[\D]+', hm_num)
            hm_num = re.sub(r'[\D]+', '', hm_num)
            salarys_1.append(hm_num)
            salarys_2.append(None)
            val.append(hm_val[0])
        elif hm2:
            hm_num = re.sub(r'\xa0', '', salarys_all[i])
            hm_num = re.sub(r'до', '', hm_num)
            hm_val = re.search(r'[\D]+', hm_num)
            hm_num = re.sub(r'[\D]+', '', hm_num)
            salarys_1.append(None)
            salarys_2.append(hm_num)
            val.append(hm_val[0])
        elif hm3:
            hm_num = re.split(r'—', salarys_all[i])
            hm_num1 = re.sub(r'\xa0', '', hm_num[0])
            hm_num2 = re.sub(r'\xa0', '', hm_num[1])
            hm_val = re.search(r'[\D]+', hm_num2)
            hm_num2 = re.sub(r'[\D]+', '', hm_num2)
            salarys_1.append(hm_num1)
            salarys_2.append(hm_num2)
            val.append(hm_val[0])
        elif hm4:
            salarys_1.append(None)
            salarys_2.append(None)
            val.append(None)
        else:
            hm_num = re.sub(r'\xa0', '', salarys_all[i])
            hm_val = re.search(r'[\D]+', hm_num)
            hm_num = re.sub(r'[\D]+', '', hm_num)
            salarys_1.append(hm_num)
            salarys_2.append(hm_num)
            val.append(hm_val[0])
                
    links_page = []
    links_page1 = html_last.findAll('a',{'class':'_1QIBo'})
    for i in links_page1:
        links_page2 = i['href']
        links_page.append(f'{global_name}{links_page2}')
    
    result['names'] = names_list
    result['salarys_1'] = salarys_1
    result['salarys_2'] = salarys_2
    result['val'] = val
    result['links'] = links_page
    result['from'] = global_name
    
    return result

In [6]:
search_s = input('Введите название вакансии: ')

Введите название вакансии: Программист


In [7]:
search = re.sub(r'\s', '+', search_s)
main_link = f'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&order_by=publication_time&text={search}&salary='

In [8]:
search_two = transliterate.translit(search_s, reversed=True)
search_two = re.sub(r'\s', '-', search_two)
main_link_two = f'https://superjob.ru/vakansii/{search_two}.html?geo%5Bt%5D%5B0%5D=4'

In [9]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
response = requests.get(main_link, headers = header).text
html = bs(response,'lxml')

In [10]:
pager = html.findAll('span',{'class':'pager-item-not-in-short-range'})
last_page = pager[-1].find('a',{'class':'bloko-button HH-Pager-Control', 'data-qa': 'pager-page'})
link_last_num = last_page.getText()

In [11]:
page_other = pd.DataFrame(columns=['names', 'salarys_1', 'salarys_2', 'val', 'links', 'from'])
for i in range(int(link_last_num)):
    link_last = f'/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&order_by=publication_time&search_field=name&text={search}&page={i}'
    result = page_table(link_last)
    page_other = pd.merge(page_other, result, how='outer')

In [12]:
response_two = requests.get(main_link_two, headers = header).text
html_two = bs(response_two,'lxml')

In [13]:
pager_two = html_two.findAll('div',{'class':'L1p51'})
last_page_two = pager_two[0].findAll('span',{'class':'_3IDf-'})
link_last_num_two = last_page_two[-2].getText()

In [14]:
page_other_two = pd.DataFrame(columns=['names', 'salarys_1', 'salarys_2', 'val', 'links', 'from'])
for i in range(1, int(link_last_num_two)+1):
    link_last = f'/vakansii/{search_two}.html?geo%5Bt%5D%5B0%5D=4&page={i}'
    result = page_table_two(link_last)
    page_other_two = pd.merge(page_other_two, result, how='outer')

In [15]:
page_other

,names,salarys_1,salarys_2,val,links,from
0,Удаленная работа Middle PHP программист (Symfo...,900,1300,USD,https://hh.ru/vacancy/35635063?query=%D0%9F%D1...,https://hh.ru
1,C# разработчик (Senior),200000,240000,руб.,https://hh.ru/vacancy/29832222?query=%D0%9F%D1...,https://hh.ru
2,Программист-разработчик (React \ ExtJS),None,None,None,https://hh.ru/vacancy/35763109?query=%D0%9F%D1...,https://hh.ru
3,Программист 1С,120000,None,руб.,https://hh.ru/vacancy/35762372?query=%D0%9F%D1...,https://hh.ru
4,Программист-разработчик python (midle),90000,200000,руб.,https://hh.ru/vacancy/35758988?query=%D0%9F%D1...,https://hh.ru
...,...,...,...,...,...,...
1987,Программист 1С,130000,170000,руб.,https://hh.ru/vacancy/35310811?query=%D0%9F%D1...,https://hh.ru
1988,Golang Developer/ Golang разработчик,None,None,None,https://hh.ru/vacancy/35684696?query=%D0%9F%D1...,https://hh.ru
1989,Front-end Developer,None,None,None,https://hh.ru/vacancy/35385029?query=%D0%9F%D1...,https://hh.ru
1990,Разработчик MsSQL и C#,160000,200000,руб.,https://hh.ru/vacancy/35744544?query=%D0%9F%D1...,https://hh.ru


In [16]:
page_other_two

,names,salarys_1,salarys_2,val,links,from
0,Инженер-программист,None,None,None,https://superjob.ru/vakansii/inzhener-programm...,https://superjob.ru
1,Ведущий программист 1С,None,150000,₽,https://superjob.ru/vakansii/veduschij-program...,https://superjob.ru
2,Программист микроконтроллеров / Программист С,None,None,None,https://superjob.ru/vakansii/programmist-mikro...,https://superjob.ru
3,Программист 1С,None,None,None,https://superjob.ru/vakansii/programmist-1s-30...,https://superjob.ru
4,JavaScript (веб) программист,100000,110000,₽,https://superjob.ru/vakansii/javascript-331377...,https://superjob.ru
...,...,...,...,...,...,...
226,DevOps Engineer / Системный администратор,None,None,None,https://superjob.ru/vakansii/devops-engineer-3...,https://superjob.ru
227,Middle Golang Developer,None,None,None,https://superjob.ru/vakansii/middle-golang-dev...,https://superjob.ru
228,Middle Android Developer,None,None,None,https://superjob.ru/vakansii/middle-android-de...,https://superjob.ru
229,Java-разработчик,None,None,None,https://superjob.ru/vakansii/java-razrabotchik...,https://superjob.ru


In [17]:
page_other_all = pd.merge(page_other, page_other_two, how='outer')

In [18]:
page_other_all

,names,salarys_1,salarys_2,val,links,from
0,Удаленная работа Middle PHP программист (Symfo...,900,1300,USD,https://hh.ru/vacancy/35635063?query=%D0%9F%D1...,https://hh.ru
1,C# разработчик (Senior),200000,240000,руб.,https://hh.ru/vacancy/29832222?query=%D0%9F%D1...,https://hh.ru
2,Программист-разработчик (React \ ExtJS),None,None,None,https://hh.ru/vacancy/35763109?query=%D0%9F%D1...,https://hh.ru
3,Программист 1С,120000,None,руб.,https://hh.ru/vacancy/35762372?query=%D0%9F%D1...,https://hh.ru
4,Программист-разработчик python (midle),90000,200000,руб.,https://hh.ru/vacancy/35758988?query=%D0%9F%D1...,https://hh.ru
...,...,...,...,...,...,...
2218,DevOps Engineer / Системный администратор,None,None,None,https://superjob.ru/vakansii/devops-engineer-3...,https://superjob.ru
2219,Middle Golang Developer,None,None,None,https://superjob.ru/vakansii/middle-golang-dev...,https://superjob.ru
2220,Middle Android Developer,None,None,None,https://superjob.ru/vakansii/middle-android-de...,https://superjob.ru
2221,Java-разработчик,None,None,None,https://superjob.ru/vakansii/java-razrabotchik...,https://superjob.ru


In [19]:
def load_mongo(page, name_mongo):
    j_page = page.to_json(orient='records')
    j_dict = json.loads(j_page)
    name_mongo.insert_many(j_dict)

In [20]:
load_mongo(page_other, vakans_hh)
load_mongo(page_other_two, vakans_sj)

### 2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы
3)*Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта. Доработать функцию, которая будет обновлять старые вакансии.

In [27]:
min_sal = input('Введите минимальный уровень зарплаты: ')

Введите минимальный уровень зарплаты: 2000


In [53]:
all_sal = vakans_hh.find({'$and':[{'salarys_1':{'$gt':min_sal}}, {'salarys_1':{'$ne': 'None'}}]}).sort('salarys_1')

In [54]:
for i in all_sal:
    pprint(i)

{'_id': ObjectId('5e413bdf7d9a9224199953d4'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35295939?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Программист 1С',
 'salarys_1': '20000',
 'salarys_2': '60000',
 'val': ' руб.'}
{'_id': ObjectId('5e413bdf7d9a92241999584f'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35646714?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Web-разработчик Python',
 'salarys_1': '20000',
 'salarys_2': '100000',
 'val': ' руб.'}
{'_id': ObjectId('5e413bdf7d9a922419995a6b'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35684451?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Back-end разработчик',
 'salarys_1': '20000',
 'salarys_2': '80000',
 'val': ' руб.'}
{'_id': ObjectId('5e4147e98623e95392437443'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35295939?query=%D0%9F%D1%80%D0%BE%D0%B3%

 'salarys_2': 'None',
 'val': ' USD'}
{'_id': ObjectId('5e4147e98623e95392437970'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35445863?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Senior C++ developer, VMS cloud',
 'salarys_1': '4000',
 'salarys_2': None,
 'val': ' USD'}
{'_id': ObjectId('5e4147e98623e95392437989'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/28163901?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Ведущий разработчик/разработчик C++/Qt (GUI Team, VMS)',
 'salarys_1': '4000',
 'salarys_2': None,
 'val': ' USD'}
{'_id': ObjectId('5e4147e98623e9539243798a'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/30005963?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Python developer',
 'salarys_1': '4000',
 'salarys_2': None,
 'val': ' USD'}
{'_id': ObjectId('5e4147e98623e95392437a04'),
 'from': 'https://hh.ru',
 'link

 'val': ' руб.'}
{'_id': ObjectId('5e4147e98623e953924378ba'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35265183?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Инженер – программист АСУТП',
 'salarys_1': '60000',
 'salarys_2': '70000',
 'val': ' руб.'}
{'_id': ObjectId('5e4147e98623e953924378c5'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35646807?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Программист C',
 'salarys_1': '60000',
 'salarys_2': '120000',
 'val': ' руб.'}
{'_id': ObjectId('5e4147e98623e953924378e3'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/34681637?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Инженер-программист АСУ ТП',
 'salarys_1': '60000',
 'salarys_2': '90000',
 'val': ' руб.'}
{'_id': ObjectId('5e4147e98623e953924378f9'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35711452?quer

 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35391998?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Разработчик Java / Java developer',
 'salarys_1': '80000',
 'salarys_2': '180000',
 'val': ' руб.'}
{'_id': ObjectId('5e4147e98623e95392437744'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35472162?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Senior Sales Manager / Business Development Manager',
 'salarys_1': '80000',
 'salarys_2': '600000',
 'val': ' руб.'}
{'_id': ObjectId('5e4147e98623e95392437779'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35631476?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Middle-Frontend разработчик (Vue js)',
 'salarys_1': '80000',
 'salarys_2': '140000',
 'val': ' руб.'}
{'_id': ObjectId('5e4147e98623e95392437787'),
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35658839?query=%D0%9